In [1]:
import physipy
from physipy.quickstart import *
import IPython
import traitlets

In [2]:
from physipy import qwidgets
import ipywidgets as ipyw

cm = units["cm"]


In [3]:
start_length = 50*cm
start_length.favunit = cm

qws = qwidgets.QuantityFloatSlider(start_length, min=0.5*m, max=10*m, step=0.1*cm)
qws

In [4]:
print(qws.value         )
print(qws.dimension     )
print(qws.favunit       )
print(qws.favunit.symbol)

0.5
L
1 m
m


In [6]:
focale = 50*cm
focale.favunit = cm
qws = qwidgets.QuantityFloatSlider(focale, min=0.5*cm, max=1*m, step=0.1*cm)


In [7]:
qws

In [12]:
print(qws.value_q)

89.0 cm


In [5]:
ipyw.FloatSlider(2)

FloatSlider(value=2.0)

In [7]:
import ipywidgets as ipyw
import physipy
import traitlets
import numpy as np
from physipy import quantify, Quantity, DimensionError, units, Dimension
from physipy.quantity.dimension import SI_UNIT_SYMBOL
pi = np.pi


q = 5*m

class QuantityWithTraits(traitlets.HasTraits):
    
    quantityFloat = traitlets.Float()
    quantityDimension = traitlets.Dict()
    
    def __init__(self, float_value, dim_dict):
        self.quantityFloat = float_value
        self.quantityDimension = dim_dict

    #@traitlets.default("quantityFloat")
    #def _get_default_quantityFloat(self):
    #    return 0.0
    #
    #@traitlets.default("quantityDimension")
    #def _get_default_quantityDimension(self):
    #    return SI_UNIT_SYMBOL
    
qwt = QuantityWithTraits(q.value, q.dimension.dim_dict)
print(q)
print(qwt)

5 m


In [8]:
class QuantityWrappedInTraits(traitlets.HasTraits):
    
    quantityTrait = traitlets.Instance(Quantity)
    
    def __init__(self, quantity):
        self.quantityTrait = quantity
        
qwit = QuantityWrappedInTraits(q)
print(qwt)

In [9]:
class QuantityWrappedInTraitspp(traitlets.HasTraits):
    
    quantityTrait = traitlets.Instance(klass=Quantity, args=(Quantity(0.0, Dimension(None)), ))
    
    def __init__(self, quantity):
        self.quantityTrait = quantity
        
    
        
qwitpp = QuantityWrappedInTraitspp(q)
print(qwitpp.quantityTrait)

5 m


In [3]:
from physipy import quantify

from traitlets import (
    Instance, Unicode, CFloat, Bool, CaselessStrEnum, Tuple, TraitError, validate, default, 
)
from traitlets.traitlets import _validate_bounds

from ipywidgets.widgets.widget_description import DescriptionWidget
from ipywidgets.widgets.trait_types import InstanceDict, NumberFormat
from ipywidgets.widgets.valuewidget import ValueWidget
from ipywidgets.widgets.widget import register, widget_serialization
from ipywidgets.widgets.widget_core import CoreWidget
from ipywidgets.widgets.widget_int import ProgressStyle



from ast import literal_eval
import contextlib
import inspect
import os
import re
import sys
import types
import enum
from warnings import warn, warn_explicit

from traitlets.utils.getargspec import getargspec
from traitlets.utils.importstring import import_item
from traitlets.utils.sentinel import Sentinel
from traitlets.utils.bunch import Bunch
from traitlets.utils.descriptions import describe, class_of, add_article, repr_type
from traitlets import TraitType

SequenceTypes = (list, tuple, set, frozenset)

# backward compatibility, use to differ between Python 2 and 3.
ClassTypes = (type,)

# exports:

__all__ = [
    "default",
    "validate",
    "observe",
    "observe_compat",
    "link",
    "directional_link",
    "dlink",
    "Undefined",
    "All",
    "NoDefaultSpecified",
    "TraitError",
    "HasDescriptors",
    "HasTraits",
    "MetaHasDescriptors",
    "MetaHasTraits",
    "BaseDescriptor",
    "TraitType",
    "parse_notifier_name",
]

# any TraitType subclass (that doesn't start with _) will be added automatically

#-----------------------------------------------------------------------------
# Basic classes
#-----------------------------------------------------------------------------


Undefined = Sentinel('Undefined', 'traitlets',
'''
Used in Traitlets to specify that no defaults are set in kwargs
'''
)



def _validate_bounds(trait, obj, value):
    """
    Validate that a number to be applied to a trait is between bounds.
    If value is not between min_bound and max_bound, this raises a
    TraitError with an error message appropriate for this trait.
    """
    
    if trait.min is not None and value < trait.min:
        raise TraitError(
            "The value of the '{name}' trait of {klass} instance should "
            "not be less than {min_bound}, but a value of {value} was "
            "specified".format(
                name=trait.name, klass=class_of(obj),
                value=value, min_bound=trait.min))
    if trait.max is not None and value > trait.max:
        raise TraitError(
            "The value of the '{name}' trait of {klass} instance should "
            "not be greater than {max_bound}, but a value of {value} was "
            "specified".format(
                name=trait.name, klass=class_of(obj),
                value=value, max_bound=trait.max))
    return value


class Float(TraitType):
    """A float trait."""

    default_value = 0.0
    info_text = 'a float'

    def __init__(self, default_value=Undefined, allow_none=False, **kwargs):
        self.min = kwargs.pop('min', -float('inf'))
        self.max = kwargs.pop('max', float('inf'))
        super(Float, self).__init__(default_value=default_value,
                                    allow_none=allow_none, **kwargs)

    def validate(self, obj, value):
        if isinstance(value, int):
            value = float(value)
        if not isinstance(value, float):
            self.error(obj, value)
        return _validate_bounds(self, obj, value)

    def from_string(self, s):
        if self.allow_none and s == 'None':
            return None
        return float(s)


class CFloat(Float):
    """A casting version of the float trait."""

    def validate(self, obj, value):
        try:
            value = float(value)
        except Exception:
            self.error(obj, value)
        return _validate_bounds(self, obj, value)


class _Float(DescriptionWidget, ValueWidget, CoreWidget):
    value = CFloat(0.0, help="Float value").tag(sync=True)

    def __init__(self, value=None, **kwargs):
        if value is not None:
            kwargs['value'] = value
        super().__init__(**kwargs)


class _BoundedFloat(_Float):
    max = CFloat(100.0, help="Max value").tag(sync=True)
    min = CFloat(0.0, help="Min value").tag(sync=True)

    @validate('value')
    def _validate_value(self, proposal):
        """Cap and floor value"""
        value = proposal['value']
        if self.min > value or self.max < value:
            value = min(max(value, self.min), self.max)
        return value

    @validate('min')
    def _validate_min(self, proposal):
        """Enforce min <= value <= max"""
        min = proposal['value']
        if min > self.max:
            raise TraitError('Setting min > max')
        if min > self.value:
            self.value = min
        return min

    @validate('max')
    def _validate_max(self, proposal):
        """Enforce min <= value <= max"""
        max = proposal['value']
        if max < self.min:
            raise TraitError('setting max < min')
        if max < self.value:
            self.value = max
        return max

    
    

from ipywidgets.widgets.widget_int import IntText, BoundedIntText, IntSlider, IntProgress, IntRangeSlider, Play, SliderStyle

@register
class FloatSlider(_BoundedFloat):
    """ Slider/trackbar of floating values with the specified range.
    Parameters
    ----------
    value : float
        position of the slider
    min : float
        minimal position of the slider
    max : float
        maximal position of the slider
    step : float
        step of the trackbar
    description : str
        name of the slider
    orientation : {'horizontal', 'vertical'}
        default is 'horizontal', orientation of the slider
    readout : {True, False}
        default is True, display the current value of the slider next to it
    readout_format : str
        default is '.2f', specifier for the format function used to represent
        slider value for human consumption, modeled after Python 3's format
        specification mini-language (PEP 3101).
    """
    _view_name = Unicode('FloatSliderView').tag(sync=True)
    _model_name = Unicode('FloatSliderModel').tag(sync=True)
    step = CFloat(0.1, allow_none=True, help="Minimum step to increment the value").tag(sync=True)
    orientation = CaselessStrEnum(values=['horizontal', 'vertical'],
        default_value='horizontal', help="Vertical or horizontal.").tag(sync=True)
    readout = Bool(True, help="Display the current value of the slider next to it.").tag(sync=True)
    readout_format = NumberFormat(
        '.2f', help="Format for the readout").tag(sync=True)
    continuous_update = Bool(True, help="Update the value of the widget as the user is holding the slider.").tag(sync=True)
    disabled = Bool(False, help="Enable or disable user changes").tag(sync=True)

    style = InstanceDict(SliderStyle).tag(sync=True, **widget_serialization)



In [9]:
from physipy import quantify, Quantity, Dimension

from traitlets import (
    Instance, Unicode, CFloat, Bool, CaselessStrEnum, Tuple, TraitError, validate, default, 
)
from traitlets.traitlets import _validate_bounds

from ipywidgets.widgets.widget_description import DescriptionWidget
from ipywidgets.widgets.trait_types import InstanceDict, NumberFormat
from ipywidgets.widgets.valuewidget import ValueWidget
from ipywidgets.widgets.widget import register, widget_serialization
from ipywidgets.widgets.widget_core import CoreWidget
from ipywidgets.widgets.widget_int import ProgressStyle


from traitlets import (
    Instance, Unicode, CFloat, Bool, CaselessStrEnum, Tuple, TraitError, validate, default, 
)
from traitlets.traitlets import _validate_bounds

from ipywidgets.widgets.widget_description import DescriptionWidget
from ipywidgets.widgets.trait_types import InstanceDict, NumberFormat
from ipywidgets.widgets.valuewidget import ValueWidget
from ipywidgets.widgets.widget import register, widget_serialization
from ipywidgets.widgets.widget_core import CoreWidget
from ipywidgets.widgets.widget_int import ProgressStyle



from ast import literal_eval
import contextlib
import inspect
import os
import re
import sys
import types
import enum
from warnings import warn, warn_explicit

from traitlets.utils.getargspec import getargspec
from traitlets.utils.importstring import import_item
from traitlets.utils.sentinel import Sentinel
from traitlets.utils.bunch import Bunch
from traitlets.utils.descriptions import describe, class_of, add_article, repr_type
from traitlets import TraitType

SequenceTypes = (list, tuple, set, frozenset)

# backward compatibility, use to differ between Python 2 and 3.
ClassTypes = (type,)

# exports:

__all__ = [
    "default",
    "validate",
    "observe",
    "observe_compat",
    "link",
    "directional_link",
    "dlink",
    "Undefined",
    "All",
    "NoDefaultSpecified",
    "TraitError",
    "HasDescriptors",
    "HasTraits",
    "MetaHasDescriptors",
    "MetaHasTraits",
    "BaseDescriptor",
    "TraitType",
    "parse_notifier_name",
]

# any TraitType subclass (that doesn't start with _) will be added automatically

#-----------------------------------------------------------------------------
# Basic classes
#-----------------------------------------------------------------------------


Undefined = Sentinel('Undefined', 'traitlets',
'''
Used in Traitlets to specify that no defaults are set in kwargs
'''
)



from traitlets import TraitType
from physipy.quantity.utils import quantity_json_encoder, quantity_json_decoder

def quantity_to_json(q, widget):
    return quantity_json_encoder(q)

def quantity_from_json(q, widget):
    return quantity_json_decoder(q)


class QuantityTrait(TraitType):
    """A trait for Quantity.
    """

    #default_value = 
    info_text = 'a Quantity object'

    def __init__(self, default_value=Undefined, allow_none=False, **kwargs):
        default_value = quantify(default_value)
        default_dim = default_value.dimension
        self.min = kwargs.pop('min', Quantity(-float('inf'),
                                              default_dim))
        self.max = kwargs.pop('max', Quantity(float('inf'),
                                             default_dim))
        super(QuantityTrait, self).__init__(default_value=default_value,
                                    allow_none=allow_none, **kwargs)
    
    def validate(self, obj, value):
        if not isinstance(value, Quantity):
            self.error(obj, value)
        return _validate_bounds(self, obj, value)

#print(QuantityTrait(q).min)
#print(QuantityTrait(q).max)
#print(QuantityTrait(4).min)
#print(type(QuantityTrait(4).min))



In [10]:
class CQuantityTrait(QuantityTrait):
    """A casting version of the QuantityTrait trait."""

    def validate(self, obj, value):
        try:
            value = quantify(value)
        except Exception:
            self.error(obj, value)
        return _validate_bounds(self, obj, value)

#print(CQuantityTrait(q).min)
#print(CQuantityTrait(q).max)
#print(CQuantityTrait(4).min)
#print(type(CQuantityTrait(4).min))
#
#cq = CQuantityTrait(q)
#print(cq)
#print(cq.default_value)

In [11]:
class _QuantityTrait(DescriptionWidget, ValueWidget, CoreWidget):
    #value = CQuantityTrait(0.0, help="Quantity value").tag(sync=True)

    def __init__(self, value=None, **kwargs):
        if value is not None:
            kwargs['value'] = value
            self.__class__.value = CQuantityTrait(value, help="Quantity value").tag(sync=True, to_json=quantity_to_json, from_json=quantity_from_json)
            self.__class__.step = CQuantityTrait(Quantity(0.1,
                                                          value.dimension),
                                                 allow_none=True, help="Minimum step to increment the value").tag(sync=True, to_json=quantity_to_json, from_json=quantity_from_json)
    
        super().__init__(**kwargs)

In [12]:
class _BoundedQuantity(_QuantityTrait):
    

    @validate('value')
    def _validate_value(self, proposal):
        """Cap and floor value"""
        value = proposal['value']
        self.max = CQuantityTrait(Quantity(100,
                                           value.dimension, help="Max value")).tag(sync=True, to_json=quantity_to_json, from_json=quantity_from_json)
        self.min = CQuantityTrait(Quantity(0, 
                                           value.dimension, help="Min value")).tag(sync=True, to_json=quantity_to_json, from_json=quantity_from_json)
        if self.min > value or self.max < value:
            value = min(max(value, self.min), self.max)
        return value

    @validate('min')
    def _validate_min(self, proposal):
        """Enforce min <= value <= max"""
        min = quantify(proposal['value'])
        if min > self.max:
            raise TraitError('Setting min > max')
        if min > self.value:
            self.value = min
        return min

    @validate('max')
    def _validate_max(self, proposal):
        """Enforce min <= value <= max"""
        max = quantify(proposal['value'])
        if max < self.min:
            raise TraitError('setting max < min')
        if max < self.value:
            self.value = max
        return max


In [13]:

from ipywidgets.widgets.widget_int import IntText, BoundedIntText, IntSlider, IntProgress, IntRangeSlider, Play, SliderStyle

@register
class QuantitySlider(_BoundedQuantity):
    """ Slider/trackbar of floating values with the specified range.
    Parameters
    ----------
    value : float
        position of the slider
    min : float
        minimal position of the slider
    max : float
        maximal position of the slider
    step : float
        step of the trackbar
    description : str
        name of the slider
    orientation : {'horizontal', 'vertical'}
        default is 'horizontal', orientation of the slider
    readout : {True, False}
        default is True, display the current value of the slider next to it
    readout_format : str
        default is '.2f', specifier for the format function used to represent
        slider value for human consumption, modeled after Python 3's format
        specification mini-language (PEP 3101).
    """
    _view_name = Unicode('FloatSliderView').tag(sync=True)
    _model_name = Unicode('FloatSliderModel').tag(sync=True)
    orientation = CaselessStrEnum(values=['horizontal', 'vertical'],
        default_value='horizontal', help="Vertical or horizontal.").tag(sync=True)
    readout = Bool(True, help="Display the current value of the slider next to it.").tag(sync=True)
    #readout_format = NumberFormat(
    #    '.2f', help="Format for the readout").tag(sync=True)
    continuous_update = Bool(True, help="Update the value of the widget as the user is holding the slider.").tag(sync=True)
    disabled = Bool(False, help="Enable or disable user changes").tag(sync=True)

    style = InstanceDict(SliderStyle).tag(sync=True, **widget_serialization)


In [16]:
# import a meter "m"
from physipy.quickstart import m, s
qs = QuantitySlider(5*m, min=0*m, max=10*m)
qs

QuantitySlider(value=<Quantity : 5 m>, step=<Quantity : 5 m>)

In [18]:

print(qs.keys)
print(qs.step)
print(qs.value)
qs.value = 10*m
print(qs.value)
# this should raise a DimensionError, and it does ! qs.value = 10*s


['_dom_classes', '_model_module', '_model_module_version', '_model_name', '_view_count', '_view_module', '_view_module_version', '_view_name', 'continuous_update', 'description', 'description_tooltip', 'disabled', 'layout', 'orientation', 'readout', 'step', 'style', 'value']
10 m
10 m
10 m


In [1]:
from physipy.quickstart import *
import json

def quantity_json_encoder(q):
    if isinstance(q, Quantity):
        return q._to_json_dict()
    else:
        type_name = q.__class__.__name__
        raise TypeError(f"Object of type '{type_name}' is not JSON seralizable")


q = 5*m
jss = json.dumps(q, default=quantity_json_encoder)
print(jss)
                        
    
    
def quantity_json_decoder(dct):
    if "Quantity" in dct:
        return Quantity(dct['value'], Dimension(dct['dim_dict']))
    return dct

                        
#jss = """
#{
#    "Quantity": true,
#    "value": 5,
#    "dim": {"L": 1, "M": 0, "T": 0, "I": 0, "theta": 0, "N": 0, "J": 0, "RAD": 0, "SR": 0}
#}
#"""
q2 = json.loads(jss, object_hook=quantity_json_decoder)
print(q2)

{"Quantity": true, "value": 5, "dim_dict": {"L": 1, "M": 0, "T": 0, "I": 0, "theta": 0, "N": 0, "J": 0, "RAD": 0, "SR": 0}}
5 m
